# Week 5: A Naive Bayes classifier

Suppose President Trump gets savvy, and realizes that people can use the Android/iPhone distinction to separate his tweets from the the tweets of aides. He starts using an iPhone too. Now, how will we distinguish tweets really authored by the President?

Well, one thing we can do is train a classifier to predict authorship using the text itself.

In [55]:
import os, csv, math, random
import pandas as pd
import numpy as np

from collections import Counter

cwd = os.getcwd()
print('Current working directory: ' + cwd + '\n')
      
relativepath = os.path.join('..', 'data', 'weekfour', 'trump.csv')
trump = pd.read_csv(relativepath)
trump.head()

Current working directory: /Users/rmorriss/Documents/datahum/code



,Unnamed: 0,Unnamed: 0.1,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,0,1,My economic policy speech will be carried live...,False,9214,NaN,2016-08-08 15:20:44,False,NaN,762669882571980801,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,3107,False,False,NaN,NaN
1,1,2,"Join me in Fayetteville, North Carolina tomorr...",False,6981,NaN,2016-08-08 13:28:20,False,NaN,762641595439190016,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,2390,False,False,NaN,NaN
2,2,3,"#ICYMI: ""Will Media Apologize to Trump?"" https...",False,15724,NaN,2016-08-08 00:05:54,False,NaN,762439658911338496,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,6691,False,False,NaN,NaN
3,3,4,"Michael Morell, the lightweight former Acting ...",False,19837,NaN,2016-08-07 23:09:08,False,NaN,762425371874557952,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,6402,False,False,NaN,NaN
4,4,5,The media is going crazy. They totally distort...,False,34051,NaN,2016-08-07 21:31:46,False,NaN,762400869858115588,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,11717,False,False,NaN,NaN


Let's simplify the Trump dataframe. We only need three columns:

1. Text of the tweet
2. Source = Android or iphone
3. A random number 0-4 that we'll use to divide the dataset into five 'folds'.


In [56]:
def trump_test(a_data_frame, rowidx):
    if 'iphone' in a_data_frame['statusSource'][rowidx]:
        return 'iphone'
    elif 'android' in a_data_frame['statusSource'][rowidx]:
        return 'android'
    else:
        return 'other'
    
tweet_text = trump['text']

source = []
fold = []
for idx in trump.index:
    source.append(trump_test(trump, idx))
    fold.append(random.sample(list(range(5)), 1)[0])
source = pd.Series(source, index = trump.index)
fold = pd.Series(fold, index = trump.index)

tdf = pd.concat([tweet_text, source, fold], axis = 1)
tdf.columns = ['text', 'source', 'fold']

# limit the dataframe to columns with either android or iphone;
# exclude 'other'
tdf = tdf[(tdf['source'] == 'android') | (tdf['source'] == 'iphone')]
tdf.head()


,text,source,fold
0,My economic policy speech will be carried live...,android,0
1,"Join me in Fayetteville, North Carolina tomorr...",iphone,2
2,"#ICYMI: ""Will Media Apologize to Trump?"" https...",iphone,0
3,"Michael Morell, the lightweight former Acting ...",android,3
4,The media is going crazy. They totally distort...,android,3


Now we need to divide the dataset into a training set and a test set. This is easily done using the "folds." We select one fold as our test set and use all the others as a training set.



In [57]:
testset = tdf[tdf['fold'] == 4]
trainingset = tdf[tdf['fold'] != 4]
print('Training set includes ' + str(trainingset.shape[0]))
print('Test set includes ' + str(testset.shape[0]))

Training set includes 1097
Test set includes 293


Now we need some basic text-wrangling functions that we've used before.

In [58]:
def tokenize(astring):
    ''' Breaks a string into words, and counts them.
    Designed so it strips punctuation and lowercases everything,
    but doesn't separate hashtags and at-signs.
    '''
    wordcounts = Counter()
    # create a counter to hold the counts
    
    tokens = astring.split()
    for t in tokens:
        word = t.strip(',.!?:;-—()<>[]/"\'').lower()
        wordcounts[word] += 1
        
    return wordcounts

def create_vocab(seq_of_strings, n):
    ''' Given a sequence of text snippets, this function
    returns the n most common words. We'll use this to
    create a limited 'vocabulary'.
    '''
    vocab = Counter()
    for astring in seq_of_strings:
        counts = tokenize(astring)
        vocab = vocab + counts
    topn = [x[0] for x in vocab.most_common(n)]
    return topn

Now we can actually write functions that build a Naive Bayes model. ```train_nb_model``` is the central function here. It calls the other two.

In [59]:
def categorize(df, rowidx):
    if df.loc[rowidx, 'source'] == 'android':
        return 'positive'
    elif df.loc[rowidx, 'source'] == 'iphone':
        return 'negative'
    else:
        print('error: neither iphone nor android')
        return 'other'

def get_priors(df):
    source_counts = df.groupby('source').count()['text']
    print(source_counts)
    positive_odds = source_counts['android'] / source_counts['iphone']
    negative_odds = source_counts['iphone'] / source_counts['android']
    return math.log(positive_odds), math.log(negative_odds)

def train_nb_model(df, p):
    vocab = create_vocab(df['text'], p)
    vocabset = set(vocab)
    # we make a set because membership-checking is faster
    # in sets; but we also hold onto the list, which is ordered
    
    positive_prior, negative_prior = get_priors(df)
    
    positive_counts = Counter()
    negative_counts = Counter()
    
    for i in df.index:
        tweet = df['text'][i]
        tweet_counts = tokenize(tweet)
        category = categorize(df, i)
        if category == 'negative':
            negative_counts = negative_counts + tweet_counts
        elif category == 'positive':
            positive_counts = positive_counts + tweet_counts
    
    # Now let's organize these Counters into a DataFrame
    
    negative = pd.Series(1, index = vocab)
    positive = pd.Series(1, index = vocab)
    # notice initializing to 1 -- Laplacian smoothing
    
    for word, count in positive_counts.items():
        if word in vocabset:
            positive[word] += count
    
    for word, count in negative_counts.items():
        if word in vocabset:
            negative[word] += count
    
    all_prob = (negative + positive) / (np.sum(negative) + np.sum(positive))
    
    negative_prob = negative / np.sum(negative)
    positive_prob = positive / np.sum(positive)
    
    # note that when we sum up the negative and positive
    # columns, we are also summing up all the Laplacian 1's
    # we initially added to them
    
    model = pd.concat([negative, positive, all_prob, 
                       negative_prob, positive_prob], axis = 1) 
        
    model.columns = ['neg', 'pos', 'all_prob', 'neg_prob', 'pos_prob']
    
    # The next step is unnecessary, and will not be found in
    # most published versions of naive Bayes. I'm providing it
    # because it may help you understand the logic of the
    # algorithm.
    
    model['neg_norm'] = negative_prob / all_prob
    model['pos_norm'] = positive_prob / all_prob
    
    
    model['log_neg'] = [math.log(x) for x in model['neg_norm']]
    model['log_pos'] = [math.log(x) for x in model['pos_norm']]
    return vocab, positive_prior, negative_prior, model

vocab, positive_prior, negative_prior, model = train_nb_model(trainingset, 1500)
model.head() 
        

source
android    598
iphone     499
Name: text, dtype: int64


,neg,pos,all_prob,neg_prob,pos_prob,neg_norm,pos_norm,log_neg,log_pos
the,160,486,0.032684,0.022287,0.038614,0.681899,1.181443,-0.382873,0.166737
to,132,243,0.018973,0.018387,0.019307,0.969115,1.017616,-0.031372,0.017463
and,79,289,0.018619,0.011004,0.022962,0.591034,1.233273,-0.525882,0.209671
in,126,188,0.015887,0.017551,0.014937,1.104775,0.940237,0.099642,-0.061623
a,95,209,0.015381,0.013233,0.016606,0.860365,1.079647,-0.150398,0.076634


Notice that we're using logarithms of the probabilities, so that we can just add them up. Our priors are logarithms, too.

In [60]:
print(positive_prior, negative_prior)

0.18098465819911314 -0.18098465819911305


Now let's write a function that applies a given model to a given testset. It will have lots of arguments.

In [61]:
pd.options.mode.chained_assignment = None

def apply_model(vocab, positive_prior, negative_prior, model, testset):
    right = 0
    wrong = 0
    vocabset = set(vocab)
    odds_pos = []
    odds_neg = []

    for i in testset.index:
        odds_positive = positive_prior
        odds_negative = negative_prior
        tweet = testset['text'][i]
        tweet_counts = tokenize(tweet)
        for word, count in tweet_counts.items():
            if word not in vocabset:
                continue
            odds_positive += model.loc[word, 'log_pos']
            odds_negative += model.loc[word, 'log_neg']
            
        if odds_positive > odds_negative:
            prediction = 'positive'
        else:
            prediction = 'negative'
        
        odds_pos.append(odds_positive)
        odds_neg.append(odds_negative)

        reality = categorize(testset, i)
        if reality != 'positive' and reality != 'negative':
            continue
        elif prediction == reality:
            right += 1
        else:
            wrong += 1

    print("Got " + str(right) + " rows right, and " + str(wrong) + " wrong.")
    accuracy = (right / (wrong + right)) * 100
    print("Accuracy was {0:.2f}%".format(accuracy))
    
    resultset = testset.copy()
    resultset['odds_positive'] = odds_pos
    resultset['odds_negative'] = odds_neg
    resultset = resultset.sort_values(by = 'odds_positive')
    
    return resultset, accuracy

newtestset, accuracy = apply_model(vocab, positive_prior, 
                         negative_prior, model, testset)

Got 239 rows right, and 54 wrong.
Accuracy was 81.57%


The ```apply_model``` function returns a version of the test set with two new columns. The dataframe is sorted by the (ascending) odds of being in the positive class, so we can find the "Trumpiest" and "least Trumpy" tweets by saying ```.tail()``` or ```.head()``` respectively.

In [41]:
newtestset.tail(20)


,text,source,fold,odds_positive,odds_negative
108,"""Little"" Michael Bloomberg, who never had the ...",android,4,2.342857,-5.525900
635,I have raised/given a tremendous amount of mon...,android,4,2.367067,-5.554729
451,"Many people are equating BREXIT, and what is g...",android,4,2.385317,-6.427663
869,"Wow, Lyin' Ted Cruz really went wacko today. M...",android,4,2.388304,-7.871770
596,Hillary Clinton is unfit to be president. She ...,android,4,2.406628,-5.365968
761,"""@FrankDallasAgg: @megynkelly @realDonaldTrump...",android,4,2.433751,-8.140402
264,"We are TRYING to fight ISIS, and now our own p...",android,4,2.472093,-6.110735
476,"I am ""the king of debt.""That has been great fo...",android,4,2.497105,-7.033512
720,Crooked Hillary Clinton looks presidential? I ...,android,4,2.609593,-7.447627
172,If Cory Booker is the future of the Democratic...,iphone,4,2.627192,-6.973521


## Exercise 1.

To start with, just play around with the functions above in order to find a value of ```p``` (number of parameters in the model) that roughly maximizes accuracy on the test set.

What accuracy do you get if you train a model on the whole ```tdf``` data frame, and also apply it to ```tdf``` as a whole?

In [62]:
vocab, positive_prior, negative_prior, model = train_nb_model(tdf, 2000)
newtestset = apply_model(vocab, positive_prior, negative_prior, model, tdf)

source
android    762
iphone     628
Name: text, dtype: int64
Got 1248 rows right, and 142 wrong.
Accuracy was 89.78%


## Exercise 2.

Write a function that *cross-validates* a modeling strategy by applying it successively to five different training sets and testing it on five different test sets.

This is called "five-fold crossvalidation."

In [65]:
def fivefold_crossvalidate(tdf, p):
    accuracies = []
    for i in range(5):
        testset = tdf[tdf['fold'] == i]
        trainingset = tdf[tdf['fold'] != i]
        vocab, positive_prior, negative_prior, model = train_nb_model(trainingset, p)
        newtestset, accuracy = apply_model(vocab, positive_prior, negative_prior, model, testset)
        accuracies.append(accuracy)
        avg_acc = sum(accuracies)/len(accuracies)
        print("The average accuracy was " + str(avg_acc))

fivefold_crossvalidate(tdf, 2000)

    


source
android    606
iphone     504
Name: text, dtype: int64
Got 227 rows right, and 53 wrong.
Accuracy was 81.07%
The average accuracy was 81.07142857142857
source
android    622
iphone     509
Name: text, dtype: int64
Got 202 rows right, and 57 wrong.
Accuracy was 77.99%
The average accuracy was 79.53185328185327
source
android    600
iphone     488
Name: text, dtype: int64
Got 237 rows right, and 65 wrong.
Accuracy was 78.48%
The average accuracy was 79.1801759185865
source
android    622
iphone     512
Name: text, dtype: int64
Got 208 rows right, and 48 wrong.
Accuracy was 81.25%
The average accuracy was 79.69763193893988
source
android    598
iphone     499
Name: text, dtype: int64
Got 239 rows right, and 54 wrong.
Accuracy was 81.57%
The average accuracy was 80.07209872521334


## Exercise 3 (probably, for homework).

Do all this for the poefic dataset, trying to distinguish poetry from fiction. Create a new notebook. Copy functions as needed in order to build a naive Bayes classifier and run five-fold crossvalidation.

How much accuracy do you get? Why do you think that accuracy is higher or lower than it was on the Trump tweet data? (You might want to inspect the data itself, using Excel or a text editor.)